In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation

from dlabsim import DLABSIM_ROOT_DIR

import sys
sys.path.append(os.path.join(DLABSIM_ROOT_DIR, "scripts/static_object"))

from dlabsim.scripts.aruco_tools.pose_from_aruco import ArucoDetector
from dlabsim.scripts.aruco_tools.pose_from_aruco import cameras_intri_dict, Tmat_marker_2_center
from dlabsim.scripts.aruco_tools.pose_from_aruco import quaternion_to_rotation_vector, rotation_vector_to_quaternion, average_quaternions

np.set_printoptions(precision=5, suppress=True, linewidth=500)

In [ ]:
camera = "x1"
save_dir = "/home/tatp/Desktop/cal"

np.set_printoptions(precision=5, suppress=True, linewidth=500)

camera_matrix = np.array(cameras_intri_dict[camera]["camera_matrix"])
dist_coeffs = np.array(cameras_intri_dict[camera]["dist_coeffs"])
cam_id = cameras_intri_dict[camera]["id"]

det = ArucoDetector(cv2.aruco.DICT_4X4_50, camera_matrix, dist_coeffs, 0.053)

img_forder = os.path.join(save_dir, "images")
img_files = sorted(os.listdir(img_forder))

In [ ]:
show = False
output_debug = True

imgs = [cv2.imread(os.path.join(img_forder, img_file)) for img_file in img_files]

mujoco_site = {
    "c2m_0" : "",
    "c2m_1" : "",
    "c2m_2" : "",
    "c2m_3" : "",
    "m2c_0" : "",
    "m2c_1" : "",
    "m2c_2" : "",
    "m2c_3" : "",
    "w2c_0" : "",
    "w2c_1" : "",
    "w2c_2" : "",
    "w2c_3" : "",
    "res" : "",
}

Tmats_cam2marker = {
    0 : [],
    1 : [],
    2 : [],
    3 : [],
}

origin_site_xml = ""

Tmats_cam2markercenter = []

world2cam_frame = None
for imgid, image in enumerate(imgs):
    img, Rmats, tvecs, ids, _ = det.detect(image)
    if len(ids):
        Tmats = []
        for i in range(len(ids)):
            Tmat_cam2marker = np.eye(4)
            Tmat_cam2marker[:3,:3] = Rmats[i]
            Tmat_cam2marker[:3,3] = tvecs[i].T[0]
            Tmats.append(Tmat_cam2marker.copy())
            Tmats_cam2marker[ids[i][0]].append(Tmat_cam2marker.copy())

            if output_debug:
                posi = Tmat_cam2marker[:3,3]
                quat_xyzw = Rotation.from_matrix(Tmat_cam2marker[:3,:3]).as_quat()
                mujoco_site["c2m_{}".format(ids[i][0])] += '<site name="c2m{}_{}" pos="{} {} {}" quat="{} {} {} {}" size="0.001" type="sphere"/>\n'.format(
                    ids[i][0], img_files[imgid].split(".")[0], posi[0], posi[1], posi[2],
                    quat_xyzw[3], quat_xyzw[0], quat_xyzw[1], quat_xyzw[2])

                Tcam_mat = np.linalg.inv(Tmat_cam2marker)
                posi = Tcam_mat[:3,3]
                quat_xyzw = Rotation.from_matrix(Tcam_mat[:3,:3]).as_quat()
                mujoco_site["m2c_{}".format(ids[i][0])] += '<site name="m2c{}_{}" pos="{} {} {}" quat="{} {} {} {}" size="0.001" type="sphere"/>\n'.format(
                    ids[i][0], img_files[imgid].split(".")[0], posi[0], posi[1], posi[2],
                    quat_xyzw[3], quat_xyzw[0], quat_xyzw[1], quat_xyzw[2])

                Tmat_cam2center = Tmat_cam2marker @ Tmat_marker_2_center[ids[i][0]]
                Tcam_mat = np.linalg.inv(Tmat_cam2center)
                posi = Tcam_mat[:3,3]
                quat_xyzw = Rotation.from_matrix(Tcam_mat[:3,:3]).as_quat()
                mujoco_site["w2c_{}".format(ids[i][0])] += '<site name="w2c_in_m{}_{}" pos="{} {} {}" quat="{} {} {} {}" size="0.001" type="sphere"/>\n'.format(
                    ids[i][0], img_files[imgid].split(".")[0], posi[0], posi[1], posi[2],
                    quat_xyzw[3], quat_xyzw[0], quat_xyzw[1], quat_xyzw[2])

        if len(ids) == 2:
            quats = []
            posis = []
            for Tmat_cam2marker in Tmats:
                posi = Tmat_cam2marker[:3,3]
                quat_xyzw = Rotation.from_matrix(Tmat_cam2marker[:3,:3]).as_quat()
                quats.append(quat_xyzw)
                posis.append(posi)
            quat_cam2markercenter = average_quaternions(quats)
            posi_cam2markercenter = np.mean(posis, axis=0)
        elif len(ids) == 1:
            Tmat_cam2center = Tmats[0] @ Tmat_marker_2_center[ids[0][0]]
            quat_cam2markercenter = Rotation.from_matrix(Tmat_cam2center[:3,:3]).as_quat()
            posi_cam2markercenter = Tmat_cam2center[:3,3]

        if show:
            cv2.imshow('Image', img)
            key = cv2.waitKey(0)
            if key == ord("q"):
                break
    else:
        print("{} nothing".format(img_files[imgid]))
        continue

cv2.destroyAllWindows()

In [ ]:
import numpy as np
from scipy.optimize import minimize

def circle_error(params, points):
    """
    计算给定圆心和半径的圆与点集的误差。
    params 是一个包含圆心坐标和半径的数组，形式为 [a, b, c, r]。
    """
    a, b, c, r = params
    error = 0
    for point in points:
        # 计算每个点到圆心的距离与半径的差的平方
        error += (np.linalg.norm(point - [a, b, c]) - r) ** 2
    return error + r**2

centers = []
radius = []
for m_id in range(4):
    points = np.array(Tmats_cam2marker[m_id])[:,:3,3]

    # 初始猜测的圆心和半径
    initial_guess = np.mean(points, axis=0).tolist() + [0.185]

    # 使用 minimize 函数进行优化
    result = minimize(circle_error, initial_guess, args=(points,), method='BFGS')

    # 输出最优圆心和半径
    if result.success:
        print("------------------- 优化成功")
        fitted_circle = result.x
        print("center = ", np.array2string(fitted_circle[:3], separator=', '))
        centers.append(fitted_circle[:3].copy())
        print("r = ", fitted_circle[3])
        radius.append(fitted_circle[3])
        err_rot = circle_error(fitted_circle, points)
        print("err = ", err_rot - fitted_circle[3]**2)
    else:
        print("优化失败。")

print(np.array(centers))

In [ ]:
print(np.array(centers))
cam2world_t = np.mean(centers, axis=0).tolist()
Tmat_cam2world = np.eye(4)
Tmat_cam2world[:3,3] = cam2world_t
print(cam2world_t)
print(Tmat_cam2world)

In [ ]:
for m_id in range(4):
    points = np.array(Tmats_cam2marker[m_id])[:,:3,3]
    es = []
    for p in points:
        es.append(abs(np.linalg.norm(cam2world_t - p) - radius[m_id]))
    print("m_id = ", m_id)
    print("err mean = ", np.mean(es))
    print("err max  = ", np.max(es))    

In [ ]:
Tmats = {
    0 : [],
    1 : [],
    2 : [],
    3 : [],
}

Tmat_cam2center = np.eye(4)
Tmat_cam2center[:3,3] = cam2world_t

assert len(Tmats_cam2marker[0]) == len(Tmats_cam2marker[1]) == len(Tmats_cam2marker[2]) == len(Tmats_cam2marker[3]) == len(img_files)
for i_pic in range(len(img_files)):
    quats = []
    for i in range(4):
        quat = Rotation.from_matrix(Tmats_cam2marker[i][i_pic][:3,:3]).as_quat()
        quats.append(quat)
    qmean = average_quaternions(quats)
    Tmat_cam2center[:3,:3] = Rotation.from_quat(qmean).as_matrix()

    rot_inv = Rotation.from_quat(qmean).inv()
    for iq, q in enumerate(quats):
        apply = True
        err_rot = np.linalg.norm((rot_inv * Rotation.from_quat(q)).as_rotvec())
        if err_rot > 0.025:
            print("--- rot error: ", i_pic, iq, err_rot)
            apply = False
        err_posi = abs(np.linalg.norm(Tmats_cam2marker[iq][i_pic][:3,3] - cam2world_t) - radius[iq])
        if err_posi > 0.006:
            print("+++ posi error: ", i_pic, iq, err_posi)
            apply = False
        if apply:
            Tmats[iq].append(np.linalg.inv(Tmats_cam2marker[iq][i_pic]) @ Tmat_cam2center)

for i in range(4):
    print(len(Tmats[i]))


In [ ]:
for i in range(4):
    quats = []
    for t in Tmats[i]:
        quat = Rotation.from_matrix(t[:3,:3]).as_quat()
        quats.append(quat)
    qmean = average_quaternions(quats)
    Tmat = np.eye(4)
    Tmat[:3,:3] = Rotation.from_quat(qmean).as_matrix()
    Tmat[:3,3] = np.mean(np.array(Tmats[i])[:,:3,3], axis=0)
    print(np.array2string(Tmat, separator=', '))

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))